In [1]:

#for a buy we want the low of the mid candle to be above the EMA 200, the RSI to be above 50, and the candle should be green and engulfing the prior candle


In [ ]:
import sys
sys.path.append("../")


In [2]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from exploration.plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [8]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True) #remember to always do this before calcing stuff if you are taking from the end, as our RSI code cares about indices and is expecting them to start from 1

In [9]:
df_an =RSI(df_an)

In [10]:
df_an = apply_patterns(df_an)

In [13]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean() #finding the 200 period exponential moving average

In [14]:
our_cols = ['time', 'mid_o', 'mid_l', 'mid_c', 'ask_c', 'bid_c','ENGULFING','direction','EMA_200']

In [15]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True) #as first 14 cant have an RSI as its a 14 average one so get rid of these
df_an.reset_index(drop=True, inplace=True)